## <7-3 학습 마친 모델을 실전 투입하기>  

학습을 마친 질의응답 모델을 인퍼런스하는 과정을 실습해 보겠습니다.

### 질문에 답하는 웹 서비스 만들기
---
이 절에서는 지문과 질문을 받아 답변하는 웹 서비스를 만들어 볼 것인데요, 지문과 질문을 각각 토큰화한 뒤 모델 입력값으로 만들고 이를 모델에 입력해 지문에서 정답이 어떤 위치에 나타나는지 확률값을 계산하게 만듭니다. 이후 약간의 후처리 과정을 거쳐 응답하게 만드는 방식입니다.

<center><그림 - 질의응답 웹 서비스></center>

<p align="center"><img src="https://i.imgur.com/I4lGm3J.jpg">  



<**1단계**> **코랩 노트북 초기화**  
이번 실습은 이전 실습과 마찬가지로 코랩에 접속한 후 **[내 드라이브에 복사]**를 진행하고 [런타임 $\rightarrow$ 런타임 유형 변경] 메뉴에서 하드웨어 가속을 사용하지 않도록 [None]을 선택합니다.

<**2단계**> **환경 설정하기**    
우선 다음 명령을 차례로 실행해 의존성 있는 패키지를 설치하고 코랩 노트북과 자신의 구글 드라이브를 연동합니다. 

In [1]:
#의존성 패키지 설치
!pip install ratsnlp

     |████████████████████████████████| 42 kB 545 kB/s 
     |████████████████████████████████| 57 kB 4.4 MB/s 
     |████████████████████████████████| 2.8 MB 10.3 MB/s 
     |████████████████████████████████| 806 kB 55.2 MB/s 
     |████████████████████████████████| 398 kB 35.0 MB/s 
     |████████████████████████████████| 636 kB 59.2 MB/s 
     |████████████████████████████████| 136 kB 45.7 MB/s 
     |████████████████████████████████| 829 kB 59.2 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
     |████████████████████████████████| 3.3 MB 40.9 MB/s 
     |████████████████████████████████| 895 kB 26.7 MB/s 
     |████████████████████████████████| 1.1 MB 41.3 MB/s 
     |████████████████████████████████| 96 kB 4.5 MB/s 
     |████████████████████████████████| 271 kB 45.3 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 144 kB 63.7 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sh

In [2]:
#구글 드라이브 연동
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


다음 코드를 실행해 각종 설정을 진행합니다.   
모든 인자는 <7-2>에서 적용한 그대로 입력해야 합니다.

In [3]:
#인퍼런스 설정
from ratsnlp.nlpbook.qa import QADeployArguments
args = QADeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-qa",
    max_seq_length=128,
    max_query_length=32,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-qa/epoch=0-val_loss=0.46.ckpt


<**3단계**> **토크나이저 및 모델 불러오기**    

다음 코드를 차례로 실행해 토크니이저를 초기화하고 앞 절에서 파인튜닝한 모델의 체크포인트를 읽어들입니다.

In [4]:
#토크나이저 로드
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

In [6]:
#체크포인트 로드
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)

다음 코드를 차례로 실행해 앞 절 파인튜닝 때 사용한 `pretrained_model_name`에 해당하는 모델의 설정값들을 읽어들이고 해당 설정값대로 BERT 모델을 초기화합니다.

In [7]:
#BERT 설정 로드
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)

In [8]:
#BERT 모델 초기화
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering(pretrained_model_config)

이어서 초기화한 BERT 모델에 체크포인트를 주입하고 모델을 평가 모드로 전환합니다.

In [9]:
#체크포인트 주입하기
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

In [10]:
#평가 모드
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

<**4단계**> **모델 출력값 만들고 후처리하기**    

인퍼런스 함수는 질문(`question`)과 지문(`context`)을 입력받아 토큰화를 수행한 뒤 `input_ids`, `attention_mask`, `token_type_ids`를 만듭니다. 이들 입력값을 파이토치 텐서 자료형으로 변환한 뒤 모델에 입력합니다. 모델 출력값은 소프트맥스 함수 적용 이전의 로짓 형태입니다.

마지막으로 모델 출력을 약간 후처리하여 정답 시작 로짓(`start_logits`)의 최댓값 위치부터 정답 끝 로짓(`end_logits`)의 최댓값 위치까지의 토큰들을 이어붙여 `pred_text`로 만듭니다.  
로직에 소프트맥스를 취하더라도 최댓값은 바뀌지 않으므로 소프트맥스 적용은 생략했습니다.

In [11]:
#인퍼런스
def inference_fn(question, context):
  if question and context:
    truncated_query = tokenizer.encode(
        question,
        add_special_tokens=False,
        truncation=True,
        max_length=args.max_query_length
    )
    inputs = tokenizer.encode_plus(
        text=truncated_query,
        text_pair=context,
        truncation="only_second",
        padding="max_length",
        max_length=args.max_seq_length,
        return_token_type_ids=True,
    )
    with torch.no_grad():
      outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})
      start_pred = outputs.start_logits.argmax(dim=-1).item()
      end_pred = outputs.end_logits.argmax(dim=-1).item()
      pred_text = tokenizer.decode(input['input_ids'][start_pred:end_pred+1])
  else:
    pred_text = ""
  return {
      'question': question,
      'context': context,
      'answer': pred_text,
  } 

<**5단계**> **웹 서비스 시작하기**   

앞 단계에서 정의한 인퍼런스 함수 `inference_fn`을 가지고 다음 코드를 실행하면 파이썬 플라스크를 활용한 웹 서비스를 띄울 수 있습니다.

In [ ]:
#웹 서비스
from ratsnlp.nlpbook.qa import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.qa.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6f23-34-74-129-102.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Apr/2022 06:35:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2022 06:36:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2022 06:36:18] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Apr/2022 06:36:50] "GET / HTTP/1.1" 200 -


웹 서비스 화면이 나오면 지문과 질문을 입력하고, <답 찾기>를 누르면 아래에 답을 구해 알려 줍니다,  
다음은 같은 지문에 각기 다른 질문을 해서 얻은 응답을 보여 줍니다.

<center><그림 - 질의응답 예시></center>

<p align="center"><img src="https://i.imgur.com/wFl0r2t.png">  


<center><그림 - 질의응답 예시></center>

<p align="center"><img src="https://i.imgur.com/uu4moTQ.png">  


#### 맺음말  

이번 장에서는 질의응답 모델을 살펴보았습니다. 실습한 데이터는 KorQuAD 1.0으로 지문이 순수 텍스트이고 길이가 대체로 짧은 편입니다. 하지만 현실의 질의응답 과제는 이보다 훨씬 어렵습니다.   
이 때문에 난도가 높은 질의응답 과제용 데이터가 제안되고 있습니다. 기존보다 지문이 길고, 표가 포함된 HTML 문서 등으로 구성된 KorQuAD 2.0이 대표적입니다.

더 나아가 지문 없이 질문만 가지고 답을 할 수 있다면 금상첨화일 겁니다. 다시 말해 입력된 지문 안에서 장답을 찾아내기보다, 정답을 포함하고 있을 법한 지문을 웹에서 알아서 잘 골라내 질문에 맞는 답을 해당 지문에서 찾는 것이 사용자가 느끼기에 더 편리한 질의응답 모델이라고 할 수 있을겁니다.

그림 출처 : ratsgo